In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"oliverli0414","key":"19df8e5218b8495a8c1ea362134c4a27"}'}

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c home-credit-default-risk
!unzip home-credit-default-risk.zip

 98% 677M/688M [00:05<00:00, 105MB/s]
100% 688M/688M [00:05<00:00, 124MB/s]
Archive:  home-credit-default-risk.zip
  inflating: HomeCredit_columns_description.csv  
  inflating: POS_CASH_balance.csv    
  inflating: application_test.csv    
  inflating: application_train.csv   
  inflating: bureau.csv              
  inflating: bureau_balance.csv      
  inflating: credit_card_balance.csv  
  inflating: installments_payments.csv  
  inflating: previous_application.csv  
  inflating: sample_submission.csv   


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
application_train = pd.read_csv('application_train.csv')[['SK_ID_CURR','TARGET']]
previous_application = pd.read_csv('previous_application.csv')

In [ ]:
print(previous_application.isna().sum())
print(previous_application['SK_ID_PREV'].nunique())
print(previous_application['SK_ID_CURR'].nunique())
print(previous_application.shape[0])
print(previous_application['CODE_REJECT_REASON'].value_counts())
print(previous_application['NAME_YIELD_GROUP'].value_counts())

SK_ID_PREV                           0
SK_ID_CURR                           0
NAME_CONTRACT_TYPE                   0
AMT_ANNUITY                     372235
AMT_APPLICATION                      0
AMT_CREDIT                           1
AMT_DOWN_PAYMENT                895844
AMT_GOODS_PRICE                 385515
WEEKDAY_APPR_PROCESS_START           0
HOUR_APPR_PROCESS_START              0
FLAG_LAST_APPL_PER_CONTRACT          0
NFLAG_LAST_APPL_IN_DAY               0
RATE_DOWN_PAYMENT               895844
RATE_INTEREST_PRIMARY          1664263
RATE_INTEREST_PRIVILEGED       1664263
NAME_CASH_LOAN_PURPOSE               0
NAME_CONTRACT_STATUS                 0
DAYS_DECISION                        0
NAME_PAYMENT_TYPE                    0
CODE_REJECT_REASON                   0
NAME_TYPE_SUITE                 820405
NAME_CLIENT_TYPE                     0
NAME_GOODS_CATEGORY                  0
NAME_PORTFOLIO                       0
NAME_PRODUCT_TYPE                    0
CHANNEL_TYPE             

In [ ]:
previous_application_sorted = pd.DataFrame()
previous_application_sorted['COUNT_LOANS']=previous_application.groupby('SK_ID_CURR')['SK_ID_PREV'].nunique()
previous_application_sorted['COUNT_CONSUMER']=previous_application[previous_application['NAME_CONTRACT_TYPE']=='Consumer loans'].groupby('SK_ID_CURR')['NAME_CONTRACT_TYPE'].count()
previous_application_sorted['COUNT_CASH']=previous_application[previous_application['NAME_CONTRACT_TYPE']=='Cash loans'].groupby('SK_ID_CURR')['NAME_CONTRACT_TYPE'].count()
previous_application_sorted['COUNT_REVOLVING']=previous_application[previous_application['NAME_CONTRACT_TYPE']=='Revolving loans'].groupby('SK_ID_CURR')['NAME_CONTRACT_TYPE'].count()
previous_application_sorted['COUNT_XNA']=previous_application[previous_application['NAME_CONTRACT_TYPE']=='XNA'].groupby('SK_ID_CURR')['NAME_CONTRACT_TYPE'].count()
previous_application_sorted[['COUNT_CONSUMER','COUNT_CASH','COUNT_REVOLVING','COUNT_XNA']]=previous_application_sorted[['COUNT_CONSUMER','COUNT_CASH','COUNT_REVOLVING','COUNT_XNA']].fillna(0)
previous_application_sorted['SUM_AMT_ANNUITY'] = previous_application.groupby('SK_ID_CURR')['AMT_ANNUITY'].sum()
previous_application_sorted['AVG_AMT_ANNUITY'] = previous_application_sorted['SUM_AMT_ANNUITY']/previous_application_sorted['COUNT_LOANS']
previous_application_sorted['SUM_AMT_APPLICATION'] = previous_application.groupby('SK_ID_CURR')['AMT_APPLICATION'].sum()
previous_application_sorted['AVG_AMT_APPLICATION'] = previous_application_sorted['SUM_AMT_APPLICATION']/previous_application_sorted['COUNT_LOANS']
previous_application_sorted['SUM_AMT_CREDIT'] = previous_application.groupby('SK_ID_CURR')['AMT_CREDIT'].sum()
previous_application_sorted['AVG_AMT_CREDIT'] = previous_application_sorted['SUM_AMT_CREDIT']/previous_application_sorted['COUNT_LOANS']
previous_application_sorted['SUM_AMT_DOWN_PAYMENT'] = previous_application.groupby('SK_ID_CURR')['AMT_DOWN_PAYMENT'].sum()
previous_application_sorted['AVG_AMT_DOWN_PAYMENT'] = previous_application_sorted['SUM_AMT_DOWN_PAYMENT']/previous_application_sorted['COUNT_LOANS']
previous_application_sorted['SUM_AMT_GOODS_PRICE'] = previous_application.groupby('SK_ID_CURR')['AMT_GOODS_PRICE'].sum()
previous_application_sorted['SUM_AMT_GOODS_PRICE'] = previous_application_sorted['SUM_AMT_GOODS_PRICE']/previous_application_sorted['COUNT_LOANS']
previous_application_sorted['COUNT_MONDAY']=previous_application[previous_application['WEEKDAY_APPR_PROCESS_START']=='MONDAY'].groupby('SK_ID_CURR')['WEEKDAY_APPR_PROCESS_START'].count()
previous_application_sorted['COUNT_TUESDAY']=previous_application[previous_application['WEEKDAY_APPR_PROCESS_START']=='TUESDAY'].groupby('SK_ID_CURR')['WEEKDAY_APPR_PROCESS_START'].count()
previous_application_sorted['COUNT_WEDNESDAY']=previous_application[previous_application['WEEKDAY_APPR_PROCESS_START']=='WEDNESDAY'].groupby('SK_ID_CURR')['WEEKDAY_APPR_PROCESS_START'].count()
previous_application_sorted['COUNT_THURSDAY']=previous_application[previous_application['WEEKDAY_APPR_PROCESS_START']=='THURSDAY'].groupby('SK_ID_CURR')['WEEKDAY_APPR_PROCESS_START'].count()
previous_application_sorted['COUNT_FRIDAY']=previous_application[previous_application['WEEKDAY_APPR_PROCESS_START']=='FRIDAY'].groupby('SK_ID_CURR')['WEEKDAY_APPR_PROCESS_START'].count()
previous_application_sorted['COUNT_SATURDAY']=previous_application[previous_application['WEEKDAY_APPR_PROCESS_START']=='SATURDAY'].groupby('SK_ID_CURR')['WEEKDAY_APPR_PROCESS_START'].count()
previous_application_sorted['COUNT_SUNDAY']=previous_application[previous_application['WEEKDAY_APPR_PROCESS_START']=='SUNDAY'].groupby('SK_ID_CURR')['WEEKDAY_APPR_PROCESS_START'].count()
previous_application_sorted[['COUNT_MONDAY','COUNT_TUESDAY','COUNT_WEDNESDAY','COUNT_THURSDAY',
                             'COUNT_FRIDAY','COUNT_SATURDAY','COUNT_SUNDAY']]=previous_application_sorted[['COUNT_MONDAY','COUNT_TUESDAY','COUNT_WEDNESDAY',
                                                                                                           'COUNT_THURSDAY','COUNT_FRIDAY','COUNT_SATURDAY','COUNT_SUNDAY']].fillna(0)
previous_application_sorted['MAX_HOUR_APPR_PROCESS_START']=previous_application.groupby('SK_ID_CURR').max('HOUR_APPR_PROCESS_START')['HOUR_APPR_PROCESS_START']
previous_application_sorted['AVG_HOUR_APPR_PROCESS_START']=previous_application.groupby('SK_ID_CURR')['HOUR_APPR_PROCESS_START'].sum()/previous_application_sorted['COUNT_LOANS']
previous_application_sorted['COUNT_LAST_APPL_PER_CONTRACT']=previous_application[previous_application['FLAG_LAST_APPL_PER_CONTRACT']=='N'].groupby('SK_ID_CURR')['FLAG_LAST_APPL_PER_CONTRACT'].count()
previous_application_sorted['COUNT_LAST_APPL_IN_DAY']=previous_application[previous_application['NFLAG_LAST_APPL_IN_DAY']==0].groupby('SK_ID_CURR')['NFLAG_LAST_APPL_IN_DAY'].count()
previous_application_sorted[['COUNT_LAST_APPL_PER_CONTRACT','COUNT_LAST_APPL_IN_DAY']]=previous_application_sorted[['COUNT_LAST_APPL_PER_CONTRACT','COUNT_LAST_APPL_IN_DAY']].fillna(0)
previous_application_sorted['AVG_RATE_DOWN_PAYMENT']=previous_application.groupby('SK_ID_CURR')['RATE_DOWN_PAYMENT'].sum()/previous_application.groupby('SK_ID_CURR')['RATE_DOWN_PAYMENT'].count()
previous_application_sorted['AVG_RATE_INTEREST_PRIMARY']=previous_application.groupby('SK_ID_CURR')['RATE_INTEREST_PRIMARY'].sum()/previous_application.groupby('SK_ID_CURR')['RATE_INTEREST_PRIMARY'].count()
previous_application_sorted['AVG_RATE_INTEREST_PRIVILEGED']=previous_application.groupby('SK_ID_CURR')['RATE_INTEREST_PRIVILEGED'].sum()/previous_application.groupby('SK_ID_CURR')['RATE_INTEREST_PRIVILEGED'].count()
previous_application_sorted['COUNT_APPROVED']=previous_application[previous_application['NAME_CONTRACT_STATUS']=='Approved'].groupby('SK_ID_CURR')['NAME_CONTRACT_STATUS'].count()
previous_application_sorted['COUNT_REFUSED']=previous_application[previous_application['NAME_CONTRACT_STATUS']=='Refused'].groupby('SK_ID_CURR')['NAME_CONTRACT_STATUS'].count()
previous_application_sorted['COUNT_CANCELED']=previous_application[previous_application['NAME_CONTRACT_STATUS']=='Canceled'].groupby('SK_ID_CURR')['NAME_CONTRACT_STATUS'].count()
previous_application_sorted['COUNT_UNUSED_OFFER']=previous_application[previous_application['NAME_CONTRACT_STATUS']=='Unused offer'].groupby('SK_ID_CURR')['NAME_CONTRACT_STATUS'].count()
previous_application_sorted['MAX_DAYS_DECISION']=previous_application.groupby('SK_ID_CURR').max('DAYS_DECISION')['DAYS_DECISION']
previous_application_sorted['AVG_DAYS_DECISION']=previous_application.groupby('SK_ID_CURR')['DAYS_DECISION'].sum()/previous_application.groupby('SK_ID_CURR')['DAYS_DECISION'].count()
previous_application_sorted['COUNT_REJECT']=previous_application[previous_application['CODE_REJECT_REASON']!='XAP'].groupby('SK_ID_CURR')['CODE_REJECT_REASON'].count()
previous_application_sorted['COUNT_REJECT_HC']=previous_application[previous_application['CODE_REJECT_REASON']=='HC'].groupby('SK_ID_CURR')['CODE_REJECT_REASON'].count()
previous_application_sorted['COUNT_REJECT_LIMIT']=previous_application[previous_application['CODE_REJECT_REASON']=='LIMIT'].groupby('SK_ID_CURR')['CODE_REJECT_REASON'].count()
previous_application_sorted['COUNT_REJECT_CLIENT']=previous_application[previous_application['CODE_REJECT_REASON']=='CLIENT'].groupby('SK_ID_CURR')['CODE_REJECT_REASON'].count()
previous_application_sorted['COUNT_REJECT_SCOFR']=previous_application[previous_application['CODE_REJECT_REASON']=='SCOFR'].groupby('SK_ID_CURR')['CODE_REJECT_REASON'].count()
previous_application_sorted['COUNT_REJECT_SCO']=previous_application[previous_application['CODE_REJECT_REASON']=='SCO'].groupby('SK_ID_CURR')['CODE_REJECT_REASON'].count()
previous_application_sorted['COUNT_REJECT_XNA']=previous_application[previous_application['CODE_REJECT_REASON']=='XNA'].groupby('SK_ID_CURR')['CODE_REJECT_REASON'].count()
previous_application_sorted['COUNT_REJECT_VERIF']=previous_application[previous_application['CODE_REJECT_REASON']=='VERIF'].groupby('SK_ID_CURR')['CODE_REJECT_REASON'].count()
previous_application_sorted['COUNT_REJECT_SYSTEM']=previous_application[previous_application['CODE_REJECT_REASON']=='SYSTEM'].groupby('SK_ID_CURR')['CODE_REJECT_REASON'].count()
previous_application_sorted[['COUNT_REJECT','COUNT_REJECT_HC','COUNT_REJECT_LIMIT','COUNT_REJECT_CLIENT','COUNT_REJECT_SCOFR',
                             'COUNT_REJECT_SCO','COUNT_REJECT_XNA','COUNT_REJECT_VERIF','COUNT_REJECT_SYSTEM']]=previous_application_sorted[['COUNT_REJECT','COUNT_REJECT_HC','COUNT_REJECT_LIMIT','COUNT_REJECT_CLIENT','COUNT_REJECT_SCOFR',
                             'COUNT_REJECT_SCO','COUNT_REJECT_XNA','COUNT_REJECT_VERIF','COUNT_REJECT_SYSTEM']].fillna(0)
previous_application_sorted['SUM_CNT_PAYMENT']=previous_application.groupby('SK_ID_CURR')['CNT_PAYMENT'].sum()
previous_application_sorted['AVG_CNT_PAYMENT']=previous_application_sorted['SUM_CNT_PAYMENT']/previous_application.groupby('SK_ID_CURR')['CNT_PAYMENT'].count()
previous_application_sorted['COUNT_YIELD_GROUP_MIDDLE']=previous_application[previous_application['NAME_YIELD_GROUP']=='middle'].groupby('SK_ID_CURR')['NAME_YIELD_GROUP'].count()
previous_application_sorted['COUNT_YIELD_GROUP_ACTION']=previous_application[previous_application['NAME_YIELD_GROUP']=='low_action'].groupby('SK_ID_CURR')['NAME_YIELD_GROUP'].count()
previous_application_sorted['COUNT_YIELD_GROUP_HIGH']=previous_application[previous_application['NAME_YIELD_GROUP']=='high'].groupby('SK_ID_CURR')['NAME_YIELD_GROUP'].count()
previous_application_sorted['COUNT_YIELD_GROUP_NORMAL']=previous_application[previous_application['NAME_YIELD_GROUP']=='low_normal'].groupby('SK_ID_CURR')['NAME_YIELD_GROUP'].count()
previous_application_sorted[['COUNT_YIELD_GROUP_MIDDLE','COUNT_YIELD_GROUP_ACTION','COUNT_YIELD_GROUP_HIGH','COUNT_YIELD_GROUP_NORMAL']]=previous_application_sorted[['COUNT_YIELD_GROUP_MIDDLE','COUNT_YIELD_GROUP_ACTION','COUNT_YIELD_GROUP_HIGH','COUNT_YIELD_GROUP_NORMAL']].fillna(0)
previous_application_sorted['COUNT_YIELD_GROUP_LOW']=previous_application_sorted['COUNT_YIELD_GROUP_ACTION']+previous_application_sorted['COUNT_YIELD_GROUP_NORMAL']
previous_application_sorted['PERC_INSURED'] = previous_application.groupby('SK_ID_CURR')['NFLAG_INSURED_ON_APPROVAL'].sum()/previous_application.groupby('SK_ID_CURR')['NFLAG_INSURED_ON_APPROVAL'].count()

In [ ]:
previous_application_sorted__application_train = previous_application_sorted.merge(application_train, on='SK_ID_CURR', how='inner')
print(previous_application_sorted__application_train)

        SK_ID_CURR  COUNT_LOANS  COUNT_CONSUMER  COUNT_CASH  COUNT_REVOLVING  \
0           100002            1             1.0         0.0              0.0   
1           100003            3             2.0         1.0              0.0   
2           100004            1             1.0         0.0              0.0   
3           100006            9             2.0         5.0              2.0   
4           100007            6             2.0         4.0              0.0   
...            ...          ...             ...         ...              ...   
291052      456251            1             1.0         0.0              0.0   
291053      456252            1             1.0         0.0              0.0   
291054      456253            2             2.0         0.0              0.0   
291055      456254            2             2.0         0.0              0.0   
291056      456255            8             3.0         4.0              1.0   

        COUNT_XNA  SUM_AMT_ANNUITY  AVG

In [ ]:
corr = previous_application_sorted__application_train.corr(method='spearman')
print(corr)

                              SK_ID_CURR  COUNT_LOANS  COUNT_CONSUMER  \
SK_ID_CURR                      1.000000     0.002829        0.002710   
COUNT_LOANS                     0.002829     1.000000        0.500635   
COUNT_CONSUMER                  0.002710     0.500635        1.000000   
COUNT_CASH                      0.001130     0.776020        0.006477   
COUNT_REVOLVING                 0.002576     0.497742        0.058534   
COUNT_XNA                      -0.003050     0.026900        0.003455   
SUM_AMT_ANNUITY                 0.003122     0.807828        0.328293   
AVG_AMT_ANNUITY                 0.000869     0.112109       -0.082736   
SUM_AMT_APPLICATION             0.003305     0.786722        0.273062   
AVG_AMT_APPLICATION             0.001670     0.263648       -0.067185   
SUM_AMT_CREDIT                  0.003358     0.796149        0.250892   
AVG_AMT_CREDIT                  0.002079     0.306506       -0.085710   
SUM_AMT_DOWN_PAYMENT            0.004377     0.2734

In [ ]:
corr.to_excel(r'previous_application.spearman.xlsx',index=True,header=True)

Choose COUNT_REVOLVING, AVG_AMT_ANNUITY, AVG_AMT_DOWN_PAYMENT, AVG_DAYS_DECISION, COUNT_REJECT, COUNT_YIELD_GROUP_HIGH, COUNT_YIELD_GROUP_LOW, due to correlation and intuitive sense to Target